In [4]:
import numpy as np
import pandas as pd
import os

In [12]:
data_path = '/kaggle/input/mit-bih/'
window_input= 200
train_size = 0.8
test_size = 1 - train_size
data_set = {
  0: "testset",
  1: "trainset"
}

In [6]:
def get_data(istrainset):    
    missing_file = 0
    total_sample = 0
    directory = f"{data_path}{data_set[istrainset]}/"
    X, y = [], []
    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        if os.path.isfile(f):
            df = pd.read_csv(f, header=None)
            data=df.drop(columns=187)
            data=data.values
            data1=df.iloc[:, 187]
            # Số lượng lặp qua dữ liệu
            num_samples = len(data) - window_input - 1 - 80
            # Tạo dữ liệu train từ cửa sổ trượt
            if num_samples > 0:
                total_sample = total_sample + num_samples
                for i in range(num_samples):
                    X_window = data[i:i+window_input]
                    y_value = data1[i+window_input+80]

                    X.append(X_window)
                    y.append(y_value)
            else:
                missing_file = missing_file + 1
    return X,y
    print(f"Num of file can not use due to its missing of length: {missing_file}")
    print(f"Number of sample: {len(y)}/{len(X)}/{total_sample}")

In [13]:
# Lấy tập train
X_train, y_train = get_data(1)

# Lấy tập test
X_test, y_test = get_data(0)

In [14]:
print(f"Len(X_train/y_train): {len(X_train)}/{len(y_train)}")
print(f"Len(X_test/y_test): {len(X_test)}/{len(y_test)}")
print(f"Test Rate: ",len(X_test)/(len(X_train)+len(X_test)))
print(f"Train Rate: ",len(X_train)/(len(X_train)+len(X_test)))

Len(X_train/y_train): 98551/98551
Len(X_test/y_test): 27984/27984
Test Rate:  0.22115620184138776
Train Rate:  0.7788437981586123


In [ ]:
X_train= np.array(X_train)
y_train= np.array(y_train)
X_test= np.array(X_test)
y_test= np.array(y_test)

In [15]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

AttributeError: 'list' object has no attribute 'shape'

In [ ]:
#X_train = X_train.reshape(X_train.shape[0], -1)
#y_train = y_train.reshape( -1)
#X_test = X_test.reshape(X_test.shape[0], -1)
#y_test = y_test.reshape(-1)

**RNN**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN

# Xây dựng mô hình RNN
model_rnn = Sequential()
model_rnn.add(SimpleRNN(64, input_shape=(200, 187)))  # Đặt kích thước đầu vào là (40, 187)
model_rnn.add(Dense(5, activation='sigmoid'))  # Đầu ra là một giá trị nhị phân

# Biên dịch mô hình
model_rnn.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


# Huấn luyện mô hình
model_rnn.fit(X_train, y_train, epochs=10, batch_size=32)

y_pred_rnn = model_rnn.predict(X_test)

In [ ]:
# Chuyển dạng xác suất thành dự đoán
y_pred_rnn = y_pred_rnn.argmax(axis=-1)

In [ ]:
# Lưu mô hình cuối cùng
model_rnn.save('model_rnn')

In [ ]:
# Đánh giá mô hình
# 1.Accuracy
from sklearn.metrics import accuracy_score
accuracy_test = accuracy_score(y_test, y_pred_rnn)
print("Độ chính xác trên dữ liệu kiểm tra RNN:", accuracy_test)

print("\n")
# 2. Precision cho từng lớp
from sklearn.metrics import precision_score
precision_scores = precision_score(y_test, y_pred_rnn, average=None, zero_division=0.0)
# In độ chính xác cho từng lớp
for i, score in enumerate(precision_scores):
    print("Precision for class {}: {}".format(i, score))

print("\n")
# 3. Recall cho từng lớp
from sklearn.metrics import recall_score
# Tính Recall trên từng lớp
recall_per_class = recall_score(y_test, y_pred_rnn, average=None)
# In kết quả
for class_idx, recall in enumerate(recall_per_class):
    print("Recall for class", class_idx, ":", recall)
    
print("\n")
# 4. F1_score
from sklearn.metrics import f1_score
f1 = f1_score(y_test, y_pred_rnn, average=None)
print("F1 score:", f1)

print("\n")
# 5. Confusion Matrix
from sklearn.metrics import confusion_matrix
confusion_mat = confusion_matrix(y_test, y_pred_rnn)
print("Confusion matrix:\n", confusion_mat)

**CNN**

In [ ]:
# Bước: Xây dựng mô hình CNN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense

input_shape = (200, 187)

model_cnn = Sequential()
model_cnn.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=input_shape))
model_cnn.add(MaxPooling1D(pool_size=2))
model_cnn.add(Conv1D(32, kernel_size=3, activation='relu'))
model_cnn.add(MaxPooling1D(pool_size=2))
model_cnn.add(Flatten())
model_cnn.add(Dense(16, activation='relu'))
model_cnn.add(Dense(5, activation='softmax'))
model_cnn.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model_cnn.fit(X_train, y_train, epochs=10, batch_size=32)

In [ ]:
y_pred_cnn = model_cnn.predict(X_test)

In [ ]:
# Chuyển dạng xác suất thành dự đoán
y_pred_cnn = y_pred_cnn.argmax(axis=-1)

In [ ]:
# Lưu mô hình cuối cùng
model_cnn.save('model_cnn')

In [ ]:
# Đánh giá mô hình
# 1.Accuracy
from sklearn.metrics import accuracy_score
accuracy_test = accuracy_score(y_test, y_pred_cnn)
print("Độ chính xác trên dữ liệu kiểm tra CNN:", accuracy_test)

print("\n")
# 2. Precision cho từng lớp
from sklearn.metrics import precision_score
precision_scores = precision_score(y_test, y_pred_cnn, average=None, zero_division=0.0)
# In độ chính xác cho từng lớp
for i, score in enumerate(precision_scores):
    print("Precision for class {}: {}".format(i, score))

print("\n")
# 3. Recall cho từng lớp
from sklearn.metrics import recall_score
# Tính Recall trên từng lớp
recall_per_class = recall_score(y_test, y_pred_cnn, average=None)
# In kết quả
for class_idx, recall in enumerate(recall_per_class):
    print("Recall for class", class_idx, ":", recall)
    
print("\n")
# 4. F1_score
from sklearn.metrics import f1_score
f1 = f1_score(y_test, y_pred_cnn, average=None)
print("F1 score:", f1)

print("\n")
# 5. Confusion Matrix
from sklearn.metrics import confusion_matrix
confusion_mat = confusion_matrix(y_test, y_pred_cnn)
print("Confusion matrix:\n", confusion_mat)